# Read Functions

Continuation of the previous notebook. Will modularize the read components into functions

In [ ]:
import os
import glob
import struct
import numpy as np

file = os.path.expanduser('~/Downloads/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat')

In [ ]:
class Object(object):
    pass

# Pulse
def read_pulse(fid, pri_struct, cpi_header):
    tmp = fid.read(32 * 4)
    n = pri_struct.unpack_from(tmp)
    if n[0] != 0x11111111 or n[1] != 0xEEEEEEEE:
        print('Error: Incorrect check codes: 0x{:08X} 0x{:08X}'.format(n[0], n[1]))
        return None
    
    # I/Q data
    def read_iq_block(align_num, num, fid):
        if align_num:
            tmp = fid.read(align_num * 4)
            raw = np.frombuffer(tmp, np.int16)
            return np.array(raw[::2][:num] + 1j * raw[1::2][:num])
        else:
            return np.empty(0)

    # Pulse
    pulse = Object()
    pulse.elevation = (n[4] & 0x3FFF) * 360 / 2 ** 14
    pulse.azimuth = (n[5] & 0x3FFF) * 180 / 2 ** 13
    pulse.ngate_long_hi = cpi_header.num_range_long_hi
    pulse.ngate_long_lo = cpi_header.num_range_long_lo
    pulse.ngate_short_hi = cpi_header.num_range_short_hi
    pulse.ngate_short_lo = cpi_header.num_range_short_lo

    # H channel data
    pulse.h_long_hi = read_iq_block(cpi_header.align_num_range_long_hi, cpi_header.num_range_long_hi, fid)
    pulse.h_long_lo = read_iq_block(cpi_header.align_num_range_long_lo, cpi_header.num_range_long_lo, fid)
    pulse.h_short_hi = read_iq_block(cpi_header.align_num_range_short_hi, cpi_header.num_range_short_hi, fid)
    pulse.h_short_lo = read_iq_block(cpi_header.align_num_range_short_lo, cpi_header.num_range_short_lo, fid)

    # V channel data
    pulse.v_long_hi = read_iq_block(cpi_header.align_num_range_long_hi, cpi_header.num_range_long_hi, fid)
    pulse.v_long_lo = read_iq_block(cpi_header.align_num_range_long_lo, cpi_header.num_range_long_lo, fid)
    pulse.v_short_hi = read_iq_block(cpi_header.align_num_range_short_hi, cpi_header.num_range_short_hi, fid)
    pulse.v_short_lo = read_iq_block(cpi_header.align_num_range_short_lo, cpi_header.num_range_short_lo, fid)

    return pulse


# CPI data header
def read_cpi_header(fid, cpi_struct):
    tmp = fid.read(32 * 4)
    n = cpi_struct.unpack_from(tmp)
    if n[0] != 0x0055AAFF or n[1] != 0xFFAA5500:
        print('Error: Incorrect check codes: 0x{:08X} 0x{:08X}'.format(n[0], n[1]))
        return None

    cpi_header = Object()
    cpi_header.check_code_1 = n[0]
    cpi_header.check_code_2 = n[1]
    cpi_header.check_counter = n[2]
    cpi_header.data_size = n[3]
    cpi_header.count = n[4]
    cpi_header.num_samples = n[5]
    cpi_header.align_num_range_short_hi = n[6]
    cpi_header.align_num_range_short_lo = n[7]
    cpi_header.align_num_range_long_hi = n[8]
    cpi_header.align_num_range_long_lo = n[9]
    cpi_header.num_range_short_hi = n[10]
    cpi_header.num_range_short_lo = n[11]
    cpi_header.num_range_long_hi = n[12]
    cpi_header.num_range_long_lo = n[13]
    
    return cpi_header

# PACK data header
def read_pack_header(fid, pack_struct):
    tmp = fid.read(32 * 4)
    n = pack_struct.unpack_from(tmp)
    if n[0] != 0x55555555 or n[1] != 0xAAAAAAAA:
        print('Error: Incorrect check codes: 0x{:08X} 0x{:08X}'.format(n[0], n[1]))
        return None

    pack_header = Object()
    pack_header.check_code_1 = n[0]
    pack_header.check_code_2 = n[1]
    pack_header.check_counter = n[2]
    pack_header.num_cpi = n[3]
    pack_header.pack_data_size = n[4]
    pack_header.division_number = n[5]
    pack_header.division_part_number = n[6]
    pack_header.mode = n[7] & 0x2 >> 1

    return pack_header

In [ ]:
fid = open(file, 'rb')

In [ ]:
pack_struct = struct.Struct('IIIIIHHI')
cpi_struct = struct.Struct('IIIIHHHHHHHHHHIII')
pri_struct = struct.Struct('IIHHHH')

In [ ]:
# Seems to have 24-byte master header
fid.seek(24, 0)

pack_header = read_pack_header(fid, pack_struct)

print('Pack check code 1: {:X}'.format(pack_header.check_code_1))
print('Pack check code 2: {:X}'.format(pack_header.check_code_2))
print('Pack check counter: {}'.format(pack_header.check_counter))
print('Number of CPI: {}'.format(pack_header.num_cpi))
print('Pack data size: {}'.format(pack_header.pack_data_size))
print('Division number: {} / {}'.format(
    pack_header.division_number, pack_header.division_part_number))
print('Observation mode: {}'.format(pack_header.mode))

In [ ]:
# h = np.zeros((npul, cpi_header.num_range_long_hi), dtype=complex)
fid.seek(24, 0)

for p in range(32):
    pack_header = read_pack_header(fid, pack_struct)
    print('Number of CPI: {}'.format(pack_header.num_cpi))

    for a in range(pack_header.num_cpi):
#         print('a = {}'.format(a))
        cpi_header = read_cpi_header(fid, cpi_struct)
        if cpi_header:
            print('    cpi: {} / {} / {}'.format(cpi_header.check_counter, cpi_header.count, cpi_header.data_size))
            for k in range(cpi_header.count):
                pulse = read_pulse(fid, pri_struct, cpi_header)
        else:
            break
        #         h[k, :] = pulse.h_long_hi